In [ ]:
%matplotlib inline

import os

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sbn

from smif.data_layer import Results
from tqdm import tqdm_notebook as tqdm

results = Results({'interface': 'local_parquet', 'dir': '../'})

sbn.set_context('talk', font_scale=0.8, rc={"lines.linewidth": 2.5})
sbn.set_palette('Paired')

In [ ]:
[mr for mr in results.list_model_runs() if 'arc_ed' in mr]

In [ ]:
results.available_results('arc_ed__baseline')

In [ ]:
results.list_sector_models('arc_ed__baseline')

In [ ]:
elec_outputs = [x for x in results.list_outputs('energy_demand_unconstrained') if 'electricity' in x]
elec_outputs

In [ ]:
res_outputs = [x for x in results.list_outputs('energy_demand_unconstrained') if 'residential' in x]
res_outputs

In [ ]:
arc_lads = gpd.read_file('../data/dimensions/lad_uk_2016-12/arc_lads.shp')
arc_lads

In [ ]:
model_run_names = [
    ('arc_ed__baseline',           'baseline',   'base',    'baseline'),
    ('arc_ed__baseline__old',      'baseline',   'old',     'baseline'),
    ('arc_ed__baseline__compact',  'baseline',   'compact', 'baseline'),
    ('arc_ed__baseline__expand',   'baseline',   'expand',  'baseline'),
    
    ('arc_ed__expansion',          'expansion',  'base',    'baseline'),
    ('arc_ed__expansion__old',     'expansion',  'old',     'baseline'),
    ('arc_ed__expansion__compact', 'expansion',  'compact', 'baseline'),
    ('arc_ed__expansion__expand',  'expansion',  'expand',  'baseline'),
    
    ('arc_ed__new-cities',         'new-cities', 'base',    'baseline'),
    ('arc_ed__new-cities__old',    'new-cities', 'old',     'baseline'),
    ('arc_ed__new-cities__compact','new-cities', 'compact', 'baseline'),
    ('arc_ed__new-cities__expand', 'new-cities', 'expand',  'baseline'),
    
    ('arc_ed__unplanned',          'unplanned',  'base',    'baseline'),
    ('arc_ed__unplanned__old',     'unplanned',  'old',     'baseline'),
    ('arc_ed__unplanned__compact', 'unplanned',  'compact', 'baseline'),
    ('arc_ed__unplanned__expand',  'unplanned',  'expand',  'baseline'),
]
model = 'energy_demand_unconstrained'
timesteps=[2015, 2020, 2030, 2050]

to_read = []
for output in res_outputs:
    for run, dwellings_scenario, floor_area_scenario, narrative in model_run_names:
        for t in timesteps:
            to_read.append((output, run, dwellings_scenario, floor_area_scenario, narrative, t))

In [ ]:
dfs = []
for output, run, dwellings_scenario, floor_area_scenario, narrative, t in tqdm(to_read):
    p = os.path.join('..', 'results', run, model, 'decision_0', 'output_{}_timestep_{}.parquet'.format(output, t))
    df = pd.read_parquet(p).reset_index()
    df = df[df.lad_uk_2016.isin(arc_lads.name)]
    df['dwellings'] = dwellings_scenario
    df['floor_area'] = floor_area_scenario
    df['timestep'] = t
    # df['narrative'] = narrative
    df = df.drop(['hourly'], axis=1) \
        .groupby(['lad_uk_2016', 'dwellings', 'floor_area', 'timestep']) \
        .sum() \
        .rename(columns={output: 'value'}) \
        .reset_index()
    df['variable'] = output
    dfs.append(df)
    
residential = pd.concat(dfs, axis=0)
residential.head()

In [ ]:
res_total = residential \
    .groupby(['dwellings', 'floor_area', 'timestep', 'variable']) \
    .sum() \
    .reset_index() \
    .rename(columns={
        'dwellings': 'Scenario',
        'timestep': 'Year',
        'variable': 'End use',
        'floor_area': 'New-build floor area'
    })

In [ ]:
sbn.relplot(
    data=res_total.rename(columns={'value':'Energy Demand (GWh)'}), 
    x='Year', y='Energy Demand (GWh)', 
    col='Scenario', row='End use', 
    hue='New-build floor area', facet_kws={'margin_titles':True}, kind='line')

In [ ]:
check_2050 = res_total[
    res_total['End use'].isin(['residential_electricity', 'residential_heat']) 
    & (res_total['Year'] == 2050)
    & (res_total['New-build floor area'] != 'old')
    ] \
    .drop(columns=['Year'])
base_2050 = check_2050[check_2050['New-build floor area'] == 'base'] \
    .drop(columns=['New-build floor area']) \
    .rename(columns={'value':'base_value'})
check_2050 = check_2050.merge(base_2050, on=['Scenario','End use'])
check_2050['perc_change'] = (check_2050.value - check_2050.base_value) / check_2050.base_value
check_2050.sort_values(by=['End use', 'New-build floor area', 'Scenario'])

In [ ]:
sbn.relplot(
    data=res_total[res_total['End use'] == 'residential_electricity'].rename(columns={'value':'Electricity Demand (GWh)'}), 
    x='Year', y='Electricity Demand (GWh)', 
    col='Scenario', 
    hue='New-build floor area', facet_kws={'margin_titles':True}, kind='line')

In [ ]:
fig = sbn.relplot(
    data=res_total[(res_total['End use'] == 'residential_electricity') & (res_total['New-build floor area'] != 'old')], 
    x='Year', y='value', 
    col='Scenario', 
    hue='New-build floor area', facet_kws={'margin_titles':True}, kind='line')
fig.set_axis_labels('Year', 'Residential electricity demand (GWh)')
fig.savefig('ed_res_elec_demand.png')

In [ ]:
sbn.relplot(
    data=res_total[res_total['End use'] == 'residential_heat'].rename(columns={'value':'Electricity Demand (GWh)'}), 
    x='Year', y='Electricity Demand (GWh)', 
    col='Scenario', 
    hue='New-build floor area', facet_kws={'margin_titles':True}, kind='line')

In [ ]:
fig = sbn.relplot(
    data=res_total[(res_total['End use'] == 'residential_heat') & (res_total['New-build floor area'] != 'old')], 
    x='Year', y='value', 
    col='Scenario', 
    hue='New-build floor area', facet_kws={'margin_titles':True}, kind='line')
fig.set_axis_labels('Year', 'Residential heat demand (GWh)')
fig.savefig('ed_res_heat_demand.png')

In [ ]:
res_total[(res_total.Year==2015) & (res_total['End use']=='residential_heat')]

## Electricity/sector

In [ ]:
model_run_names = [
    ('arc_ed__baseline',           'baseline',   'base',    'baseline'),
    ('arc_ed__baseline__old',      'baseline',   'old',     'baseline'),
    
    ('arc_ed__expansion',          'expansion',  'base',    'baseline'),
    ('arc_ed__expansion__old',     'expansion',  'old',     'baseline'),
    
    ('arc_ed__new-cities',         'new-cities', 'base',    'baseline'),
    ('arc_ed__new-cities__old',    'new-cities', 'old',     'baseline'),
    
    ('arc_ed__unplanned',          'unplanned',  'base',    'baseline'),
    ('arc_ed__unplanned__old',     'unplanned',  'old',     'baseline'),
]
model = 'energy_demand_unconstrained'
timesteps=[2015, 2020, 2030, 2050]

to_read = []
for output in elec_outputs:
    for run, dwellings_scenario, floor_area_scenario, narrative in model_run_names:
        for t in timesteps:
            to_read.append((output, run, dwellings_scenario, floor_area_scenario, narrative, t))

In [ ]:
dfs = []
for output, run, dwellings_scenario, floor_area_scenario, narrative, t in tqdm(to_read):
    p = os.path.join('..', 'results', run, model, 'decision_0', 'output_{}_timestep_{}.parquet'.format(output, t))
    df = pd.read_parquet(p).reset_index()
#     df = df[df.lad_uk_2016.isin(arc_lads.name)]
    df['dwellings'] = dwellings_scenario
    df['floor_area'] = floor_area_scenario
    df['timestep'] = t
    df = df.drop(['hourly'], axis=1) \
        .groupby(['lad_uk_2016', 'dwellings', 'floor_area', 'timestep']) \
        .sum() \
        .rename(columns={output: 'value'}) \
        .reset_index()
    df['variable'] = output
    dfs.append(df)
    
elec_base_old = pd.concat(dfs, axis=0)
elec_base_old.head()

In [ ]:
len(elec_base_old.lad_uk_2016.unique())

In [ ]:
sbn.relplot(
    data=elec_base_old, 
    x='timestep', 
    y='value', 
    col='dwellings', 
    row='floor_area',
    hue='variable',
    kind='line')

In [ ]:
sbn.relplot(
    data=elec_base_old.groupby(['dwellings', 'floor_area', 'timestep', 'variable']).sum().reset_index(), 
    x='timestep', 
    y='value', 
    col='dwellings', 
    row='floor_area',
    hue='variable',
    kind='line')

# Electricity/multi-vector

In [ ]:
model_run_names = [
    ('arc_ed__baseline',       'baseline',   'base'),
    ('arc_ed__baseline__ew',   'baseline',   'ew'  ),
    ('arc_ed__baseline__mv',   'baseline',   'mv'  ),
    
    ('arc_ed__expansion',      'expansion',  'base'),
    ('arc_ed__expansion__ew',  'expansion',  'ew'  ),
    ('arc_ed__expansion__mv',  'expansion',  'mv'  ),
    
    ('arc_ed__new-cities',     'new-cities', 'base'),
    ('arc_ed__new-cities__ew', 'new-cities', 'ew'  ),
    ('arc_ed__new-cities__mv', 'new-cities', 'mv'  ),
    
    ('arc_ed__unplanned',      'unplanned',  'base'),
    ('arc_ed__unplanned__ew',  'unplanned',  'ew'  ),
    ('arc_ed__unplanned__mv',  'unplanned',  'mv'  ),
]
model = 'energy_demand_unconstrained'
timesteps=[2015, 2020, 2030, 2050]

to_read = []
for output in results.list_outputs('energy_demand_unconstrained') :
    for run, dwellings_scenario, narrative in model_run_names:
        for t in timesteps:
            to_read.append((output, run, dwellings_scenario, narrative, t))

In [ ]:
results.list_outputs('energy_demand_unconstrained') 

In [ ]:
dfs = []
for output, run, dwellings_scenario, narrative, t in tqdm(to_read):
    p = os.path.join('..', 'results', run, model, 'decision_0', 'output_{}_timestep_{}.parquet'.format(output, t))
    df = pd.read_parquet(p).reset_index()
    df = df[df.lad_uk_2016.isin(arc_lads.name)]
    df['dwellings'] = dwellings_scenario
    df['narrative'] = narrative
    df['timestep'] = t
    df = df.drop(['hourly'], axis=1) \
        .groupby(['lad_uk_2016', 'dwellings', 'narrative', 'timestep']) \
        .sum() \
        .rename(columns={output: 'value'}) \
        .reset_index()
    df['variable'] = output
    dfs.append(df)
    
narrative_data = pd.concat(dfs, axis=0)
narrative_data.head()

In [ ]:
electricity = narrative_data[narrative_data.variable.str.contains('electricity')]

In [ ]:
df = narrative_data.drop('hourly', axis=1).groupby(['dwellings', 'narrative', 'timestep', 'variable']).sum().reset_index()
sbn.relplot(
    data=df, 
    x='timestep', y='value', 
    col='dwellings', row='variable', 
    hue='narrative', facet_kws={'margin_titles':True}, kind='line')

In [ ]:
by_sector = narrative_data.drop('hourly', axis=1).groupby(['lad_uk_2016', 'dwellings', 'narrative', 'timestep', 'variable']).sum().reset_index()
by_sector = by_sector[by_sector.timestep == 2050]
sbn.relplot(
    data=by_sector, 
    x='dwellings', 
    y='value', 
    hue='lad_uk_2016',
    col='variable',
    col_wrap=3
)

In [ ]:
total_elec_demand = electricity.drop('hourly', axis=1).groupby(['dwellings', 'narrative', 'timestep']).sum().reset_index()

In [ ]:
sbn.relplot(
    data=total_elec_demand, 
    x='timestep', 
    y='value', 
    col='dwellings', 
    hue='narrative',
    kind='line')

In [ ]:
sbn.relplot(
    data=electricity.drop('hourly', axis=1).groupby(['dwellings', 'narrative', 'timestep', 'variable']).sum().reset_index(), 
    x='timestep', 
    y='value', 
    col='dwellings', 
    hue='variable',
    kind='line')

# Arc LADs: Peak demand

In [ ]:
model_run_names = [
    ('arc_ed__baseline',   'baseline'  ),
    ('arc_ed__expansion',  'expansion' ),
    ('arc_ed__new-cities', 'new-cities'),
    ('arc_ed__unplanned',  'unplanned' ),
]
model = 'energy_demand_unconstrained'
timesteps=[2015, 2020, 2030, 2050]

to_read = []
for output in elec_outputs:
    for run, dwellings_scenario in model_run_names:
        for t in timesteps:
            to_read.append((output, run, dwellings_scenario, t))

In [ ]:
dfs = []
for output, run, dwellings_scenario, t in tqdm(to_read):
    p = os.path.join('..', 'results', run, model, 'decision_0', 'output_{}_timestep_{}.parquet'.format(output, t))
    df = pd.read_parquet(p).reset_index()
    df['in_arc'] = df.lad_uk_2016.isin(arc_lads.name)
    df['dwellings'] = dwellings_scenario
    df['timestep'] = t
    df['variable'] = output
    df = df.rename(columns={output: 'value'})
    dfs.append(df)
    
elec_data = pd.concat(dfs, axis=0)
elec_data.head()

In [ ]:
del dfs

In [ ]:
pdf = elec_data.groupby(['hourly', 'dwellings', 'timestep']).sum()

In [ ]:
pdf.reset_index().groupby(['dwellings','timestep']).sum()[['value']]

In [ ]:
df = pdf.reset_index().groupby(['dwellings','timestep']).max()[['value']]
df.to_csv('peak_national.csv')
df

In [ ]:
arc_pdf = elec_data[elec_data.in_arc].groupby(['hourly', 'dwellings', 'timestep']).sum()

In [ ]:
arc_pdf.reset_index().groupby(['dwellings','timestep']).sum()[['value']]

In [ ]:
df = arc_pdf.reset_index().groupby(['dwellings','timestep']).max()[['value']]
df.to_csv('peak_arc_only.csv')
df

In [ ]:
# plus 4 days skips us forwards to a Monday (weekend/holiday effect)
filtered = elec_data[
    elec_data['hourly'].isin(range(24*4, 24*4+24*7)) 
    & (elec_data.timestep == 2020)
    & (elec_data.dwellings == 'baseline')
    & elec_data.in_arc
].copy()
filtered.variable = filtered.variable.apply(lambda d: d.replace("_electricity", "").title())
# filtered.dwellings = filtered.dwellings.apply(lambda d: d.title())
filtered.rename(columns={
    'timestep': 'Year', 
    'variable': 'Sector',
    'dwellings': 'Scenario',
    'hourly': 'Hour of year',
    'value': 'Electricity demand in Arc LADs (GW)'
}, inplace=True)
fig = sbn.relplot(
    data=filtered, 
    x='Hour of year', 
    y='Electricity demand in Arc LADs (GW)', 
    col='Sector', 
#     row='Scenario', 
    hue='Sector',
    facet_kws={'margin_titles':True},
    kind='line')
fig.savefig('ed_electricity_week.png')

In [ ]:
def highlight(data, id_col='desc'):
    def group_hlads(index):
        named_lads = {
            'Aylesbury Vale', 
            'Bedford', 
            'Cambridge', 
            'Central Bedfordshire', 
            'Huntingdonshire', 
            'Milton Keynes', 
            'Oxford', 
            'South Cambridgeshire'
        }
        if index in named_lads:
            return index
        else:
            return 'Other'
    
    data['summary_lads'] = data[id_col].apply(group_hlads) 
    
    return data

In [ ]:
elec_data.columns

In [ ]:
arc_lads.head(1)

In [ ]:
elec_sum = elec_data.groupby(['dwellings', 'timestep', 'lad_uk_2016', 'hourly']).sum().reset_index()

In [ ]:
elec_in_arc = elec_sum.merge(
    arc_lads.drop(columns=['geometry','in_arc']),
    left_on='lad_uk_2016',
    right_on='name'
)
elec_in_arc = highlight(elec_in_arc)[['summary_lads', 'lad_uk_2016', 'dwellings','timestep', 'hourly', 'value']]

In [ ]:
peak = elec_in_arc.groupby(['lad_uk_2016','summary_lads','dwellings','timestep']) \
    .max() \
    .reset_index() \
    .rename(columns={
        'dwellings': 'Scenario',
        'summary_lads': 'Local Authority District'
    }) \
    .drop(columns='hourly') \
    .sort_values(by=['Local Authority District', 'Scenario'])

In [ ]:
fig = sbn.relplot(
    data=peak, 
    x='timestep', 
    y='value', 
    col='Scenario', 
    hue='Local Authority District', 
    kind='line')
fig.set_axis_labels('Year', 'Peak demand (GW)')
fig.savefig('ed_peak_demand.png')

# Arc LADs Total Demand

In [ ]:
total_elec_dem = elec_in_arc \
    .drop(columns='hourly') \
    .groupby(by=['dwellings', 'timestep', 'lad_uk_2016', 'summary_lads']) \
    .sum() \
    .reset_index() \
    .rename(columns={
        'value': 'total_demand',
        'summary_lads': 'Local Authority District',
        'dwellings': 'Scenario'
    }) \
    .sort_values(by=['Local Authority District', 'Scenario'])
total_elec_dem.head(2)

In [ ]:
fig = sbn.relplot(
    data=total_elec_dem, 
    x='timestep', 
    y='total_demand', 
    col='Scenario', 
    hue='Local Authority District', 
    kind='line'
)
fig.set_axis_labels('Year', 'Total electricity demand (GWh)')
fig.savefig('ed_total_demand.png')